In [53]:
import json
import boto3

import sys
import utils.runReport_Single_month
del sys.modules['utils.runReport_Single_month']
import utils.runReport_Single_month

import utils.productionAnalysis
del sys.modules['utils.productionAnalysis']
import utils.productionAnalysis

import utils.leapDispatchSummary

import pandas as pd
import calendar
import numpy as np
import datetime as dt
import requests

In [54]:
## Database credentials
with open("creds.json") as f:
    data=f.read()
js = json.loads(data)

client = boto3.client('s3')
# community = 'Quail Ridge'
community = 'High Desert Villas'

start_date_str = '2024-05-01'

# street_adddress = '409 E THORNTON AVE'
street_adddress = '16850 JASMINE ST'

with open("credentials_leap.json", "r") as f:
    leap_creds_dict = json.load(f)

# leap_account_id
account_id = "050b8f48-2c38-4fd7-8368-cd14fa5d07f8"
api_key = leap_creds_dict['api-key']

headers = {
    "accept": "application/json", 
    "authorization": "Bearer " + api_key,
    "content-type": "application/json"
}

meters_df = utils.leapDispatchSummary.getMeterInfo(headers, account_id)

# Get Access Token
url = 'https://flextrons.io/api/v1/auth/login'

headers = {'accept': 'application/json','Content-Type': 'application/json'}
data = {"email": "{username}", "password":"{password}"}
px_auth = requests.post(url, headers = headers, data = json.dumps(data)).json()

start_date_str = '2024-05-01'
end_date_str = '2024-05-21'

subscriber_df, rate_table_df, dispatch_summary,production_data_summary, cooling_degrees_df, eventPerformanceSummary_df = utils.runReport_Single_month.create_rev_metric_report(community, start_date_str, end_date_str, client, leap_creds_dict, account_id, js, px_auth, street_adddress, meters_df)

/Users/timmac/Desktop/git/pearlx-revenue-reporting/utils/productionAnalysis.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  commission_occupied_df['move_in'] = pd.to_datetime(commission_occupied_df['move_in'])
/Users/timmac/Desktop/git/pearlx-revenue-reporting/utils/productionAnalysis.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  commission_occupied_df['move_out'] = pd.to_datetime(commission_occupied_df['move_out'])


KeyboardInterrupt: 

In [394]:
eventPerformanceSummary_df

,values
total Target Event Energy [kWhdc],1706.812500
total Actual Event Energy [kWhdc],3303.761195
total Events [#],211.000000
unique systems dispatched [#],16.000000


In [326]:
subscriber_df

Month,May 2024
Starting Units Commissioned,137
Ending Units Commissioned,137
Commissioned & Occupied Days,889
Commissioned Days,959
Days with Units on CARE,560
Days with Units Not on CARE,329
% on CARE,63%
Occupancy,93%


In [327]:
dispatch_summary

,event_energy_wh,baseline_wh,performance_wh,dispatch_quantity_wh,number_of_events,meter_id,service_address_full
0,180.0,749.76,569.76,2100,3,ad08a538-536f-4b93-9254-3dda297d6e50,"16850 JASMINE ST APT X5, 92395, VICTORVILLE, C..."


In [328]:
production_data_summary.round(2).replace({np.nan:''})

,W Mid Peak,W Off Peak,W Super Off,units_included,Missing Intervals,Units Evaluated
bessPVOccupied,890.34,79.33,890.71,18.0,,
bessPVVacant,0.84,2.31,29.06,1.0,,
Data Metrics,,,,,2694.0,19.0


In [14]:
cooling_degrees_df

,period,seriesId,seriesDescription,value,unit
0,2024-04,ZWCD_PAC,Cooling Degree Days Pacific,7.4514974208,total degree days
1,2024-04,ZWCD_PAC_10YR,"Cooling Degree Days Pacific, Prior 10-Year Ave...",23.582566561,total degree days


In [37]:
df = pd.read_csv('allproductiondata.csv',index_col = 0)

In [45]:
working_df = df.copy()

btm_pv = working_df[['unit_number','vnem_btm']].groupby(by = ['unit_number','vnem_btm']).count().reset_index()

working_df['timestamp_start'] = pd.to_datetime(working_df['timestamp_start'])
working_df['timestamp_start_hour'] = working_df['timestamp_start'].dt.floor('H')

occflag_daily = working_df[['unit_number', 'timestamp_start_hour','occFlag']].groupby(by = ['unit_number','timestamp_start_hour']).max().reset_index()
group_df = working_df[['unit_number','timestamp_start_hour','Wh']].groupby(by = ['unit_number','timestamp_start_hour']).sum().reset_index()


In [49]:
working_group_df = group_df.merge(occflag_daily, left_on = ['unit_number','timestamp_start_hour'], right_on = ['unit_number','timestamp_start_hour'], how = 'left')
working_group_df = working_group_df.merge(btm_pv, left_on = ['unit_number'], right_on = ['unit_number'] , how = 'left')
working_group_df['date'] = working_group_df['timestamp_start_hour'].dt.date
working_group_df['hour'] = working_group_df['timestamp_start_hour'].dt.hour
working_group_df['midpeakFlag'] = 0
working_group_df.loc[(working_group_df['hour'] >= 16) & (working_group_df['hour']<= 20),'midpeakFlag'] = 1

In [51]:
working_group_df.to_csv('allProductionDataHourly.csv')